## **Turi Recommender**

Welcome to the recommendation engine model quickstart on Skafos! The purpose of this notebook is to get you going end-to-end. Below we will do the following:

1. Load movie ratings data.
2. Build a recommendation engine model.
3. Convert the model to CoreML format and save it to the Skafos framework.

The example is based on [Turi Create's Recommendation Engine](https://apple.github.io/turicreate/docs/userguide/recommender).

---

Execute each cell one-by-one, by selecting the cell and do one of the following:

-  Clicking the "play" button at the top of this frame.
-  Typing 'Control + Enter' or 'Shift + Enter'.

In [ ]:
# Import necessary libraries
from skafossdk import *

import utilities.save_models as sm
from utilities.load_turicreate import *

In [ ]:
tc = install_turicreate(timeout=500, retries=2) # check to make sure turicreate is installed
ska = Skafos() # initialize Skafos

### 1. **Load the data**

The training data for this example are movies (items) and ratings by users on those movies (actions). First, we fetch training data from an AWS S3 bucket. Then, we downsample the original data, keeping only 20% of the users for modeleing (This is done to simplify the model and minimuze time and space required. After downsampling, the remaining sampled data is randomly split into train and test sets, where 80% of the data is used for training, and 20% is used for model evaluation.


In [ ]:
# Load data from S3 bucket
actions = tc.SFrame.read_csv(
    url='https://s3.amazonaws.com/skafos.example.data/MovieLensDataset/ml-20m/ratings.csv',
    verbose=False
)

items = tc.SFrame.read_csv(
    url='https://s3.amazonaws.com/skafos.example.data/MovieLensDataset/ml-20m/movies.csv',
    verbose=False
)

In [ ]:
# Sample ~20% of the user ids to make our data smaller for this example
sample_user_ids = actions['userId'].unique().sample(.2)

# Filter the dataframe
actions = actions[actions['userId'].is_in(sample_user_ids)][['userId', 'movieId']]

In [ ]:
# Make a train-test split - this may take a couple minutes
train_data, test_data = tc.recommender.util.random_split_by_user(
    dataset=actions,
    user_id='userId',
    item_id='movieId',
)

In [ ]:
# Take a look at the data
train_data.head()

### 2. **Build the model**
We use the `tc.recommender.create` function (default) and specify the data, user id, and item id to properly train the model. To understand more about this specific function, check out the [Turi Create Documentation](https://apple.github.io/turicreate/docs/userguide/recommender/).

The model in this example is an implicit recommendation engine that trains on interaction data **without** explicit ratings or sentiment information. To see an example of an explicit recommendation engine, check out the `advanced_usage/` section.

In [ ]:
# Train the default recommender
model = tc.recommender.create(
    observation_data=train_data,
    user_id='userId',
    item_id='movieId'
)

In [ ]:
# Make a recommendation to a known user within the test set
# k ~ is the number of recommendations to generate
model.recommend(
    users=[test_data['userId'][0]],
    k=5
)

In [ ]:
# Make a recommendation to a new (unknown) user - returns most popular movies
model.recommend(
    users=[9999999999],
    k=5
)

### 3. **Save the model**
Once your model has been created, it must be saved to the Skafos framework via the code below. This will trigger a push to your mobile app.

In [ ]:
# Export to coreml
coreml_model_name = "Recommender.mlmodel"
res = model.export_coreml(coreml_model_name)

# Compress the model
compressed_model = sm.compress_model(coreml_model_name)

# Save to Skafos
sm.skafos_save_model(
    skafos=ska,
    model_name=coreml_model_name,
    compressed_model=compressed_model,
    permissions='public'
)